<a href="https://colab.research.google.com/github/natrask/ENM5320/blob/main/Code/FEMconvergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Verification of the finite element problem convergence rate for Dirichlet + Neumann boundary conditions

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch

import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Check if CUDA is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

First we'll generate our finite element grid. I'm adding noise to the nodes - if you play with the magnitude of the noise you'll see that the theory still holds.

In [ ]:

# %%
# Define points and h as needed
meshsize= 8
h = 1./float(meshsize-1)
points = torch.tensor(np.linspace(0,1,meshsize), dtype=torch.float64)
finepoints = torch.tensor(np.linspace(0,1,21*meshsize), dtype=torch.float64)

def evalPhi_i(x):
    x_expanded = torch.unsqueeze(x, 0)
    points_expanded = torch.unsqueeze(points, 1)
    return torch.relu(1.0 - (torch.abs(x_expanded - points_expanded)) / h)

def evalGradPhi_i(x):
    suppPhi = (evalPhi_i(x) > 0).double()
    signPlus = (torch.unsqueeze(points, 1) > torch.unsqueeze(x, 0)).double()
    signNeg = (torch.unsqueeze(points, 1) <= torch.unsqueeze(x, 0)).double()
    return suppPhi * (-signPlus + signNeg) / h

# Visualize shape functions and their derivatives evaluated over a fine grid
phi_i = evalPhi_i(finepoints)
grad_phi_i = evalGradPhi_i(finepoints)

plt.plot(finepoints.numpy(), phi_i.numpy().T)
plt.title("Phi_i")
plt.show()
plt.figure()
plt.plot(finepoints.numpy(), grad_phi_i.numpy().T)
plt.title("Grad Phi_i")
plt.show()

Next we'll construct the stiffness matrices. To do this, we construct a two-point [Gauss-Legendre quadrature rule](https://en.wikipedia.org/wiki/Gauss%E2%80%93Legendre_quadrature), and use it to build the stiffness matrix

$$S_{ij} = \int \phi_i' \phi_j' dx$$

In [ ]:

# %% Get Quadrature points
xql = points[:-1].numpy() + h * (0.5 + 1. / (2. * np.sqrt(3)))
xqr = points[:-1].numpy() + h * (0.5 - 1. / (2. * np.sqrt(3)))
xq = np.concatenate([xql, xqr])
xq = np.sort(xq)
# Convert xq back to a tensor
xq = torch.tensor(xq, dtype=torch.float64)

# %% Construct matrices
nodal_basisEval = evalPhi_i(xq)
nodal_gradbasisEval = evalGradPhi_i(xq)
Snodal = (h/2)*torch.einsum('ijq->ij', torch.unsqueeze(nodal_gradbasisEval, 0) * torch.unsqueeze(nodal_gradbasisEval, 1))

Finally we build up the matrix to solve. For the Dirichlet boundary node, we eliminate the corresponding equation, and overwrite with the condition:
$$u(0) = u_D(0) = 0$$

In [ ]:

# %% Construct discretization for Poisson with Dirichlet BCs on left and Neumann on right

# Set up forcing function evaluated on the nodes and specify dirichlet conditions
forcingvec = torch.ones(meshsize, dtype=torch.float64)
forcing = (h/2)*torch.einsum('i,iq->i',forcingvec,nodal_basisEval)

uLHS = 0.0

# %% Build matrices
solution_rhs = torch.cat([torch.tensor([uLHS], dtype=torch.float64), forcing[1:meshsize]], dim=0)
solution_mat = torch.cat([
    torch.unsqueeze(torch.nn.functional.one_hot(torch.tensor(0), meshsize).double(), 0),
    Snodal[1:meshsize, :]
], dim=0)


# %% Solve the linear system and plot solution
u_sol = torch.linalg.solve(solution_mat, solution_rhs)
uexact = finepoints.numpy()*(-0.5*finepoints.numpy()+1.0)
plt.plot(points.numpy(), u_sol.numpy(),'.--',label='Computed',markersize=20)
plt.plot(finepoints.numpy(), uexact,label='Exact')
plt.legend()
plt.title("Solution")
plt.show()

# Plot derivative of solution on quadrature points
u_sol_grad = torch.matmul(evalGradPhi_i(xq).T, u_sol)
plt.plot(xq.numpy(), u_sol_grad.numpy(),'.--',label='Computed',markersize=20)
uexact_grad = -1 + finepoints.numpy()
plt.plot(finepoints.numpy(), uexact_grad,label='Exact')
plt.legend()
plt.title("Solution Gradient")
plt.show()

Finally, lets compare this to the true solution and verify the derivation in class that
$$ || u - u_h || \leq C h^2 ||f|| $$
$$ || u - u_h ||_E \leq C h ||f|| $$

We can compare against the true solution:
$$u(x) = x (1-x/2)$$
$$u'(x) = 1 - x $$

In [ ]:
# Evaluate uexact and uexact' on quadrature points
uex_q = xq.numpy()*(-0.5*xq.numpy()+1.0)
uex_qprime = -1.+xq.numpy()
# Evaluate solution and solution' on quadrature points
uq = torch.matmul(nodal_basisEval.T, u_sol)
uqprime = torch.matmul(nodal_gradbasisEval.T, u_sol)
# Compute L2 error
L2error = np.sqrt(h * torch.sum((uq - uex_q) ** 2))
L2errorprime = np.sqrt(h * torch.sum((uqprime - uex_qprime) ** 2))
print(f"h/L2 error: {h},{L2error}")
print(f"h/L2 error prime: {h},{L2errorprime}")

Here is some output for rerunning the code with 4,8,16,32 nodes
|       h              | L2err      | H1err      |
|----------------------|------------|------------|
| 0.3333333333333333   | 0.013094570021973223  | 0.13608276348795434  |
| 0.14285714285714285  | 0.0024051251060773294  | 0.05832118435198045  |
| 0.06666666666666667  | 0.000523782800877644  | 0.027216552697590875  |